In [62]:
import pulp
from collections import namedtuple, defaultdict

In [63]:
prob = pulp.LpProblem("PlusPuzzle", pulp.LpMinimize)
prob += 0

In [64]:
size = 5

numbers = range(1, size + 1) # [1, 6)
xs = range(1, size + 1)
ys = range(1, size + 1)

choices = pulp.LpVariable.dicts("Cell", (xs, ys, numbers), 0, 1, pulp.LpInteger)

In [65]:
# 1つのマスに入る値は1つだけ

for y in xs:
    for x in ys:
        prob += pulp.lpSum([choices[v][x][y] for v in numbers]) == 1

In [66]:
for v in numbers:
    for y in ys:
        prob += pulp.lpSum([choices[v][x][y] for x in xs]) == 1

for v in numbers:
    for x in xs:
        prob += pulp.lpSum([choices[v][x][y] for y in ys]) == 1


In [67]:
# 問題のブロックの定義

Block = namedtuple("Block", ("sum_number", "positions"))

blocks = [
    Block(4, [(1, 1), (2, 1), (1, 2)]),
    Block(24, [(3, 1), (4, 1), (5, 1), (2, 2), (3, 2), (4, 2)]),
    Block(5, [(5, 2), (5, 3), (4, 3)]),
    Block(7, [(2, 3), (3, 3)]),
    Block(12, [(1, 3), (1, 4), (1, 5)]),
    Block(5, [(2, 4), (2, 5)]),
    Block(4, [(3, 4), (3, 5), (4, 5)]),
    Block(14, [(4, 4), (5, 4), (5, 5)]),
]

In [68]:
def validate_blocks(blocks):
    d = defaultdict(lambda: 0)
    for b in blocks:
        for p in b.positions:
            d[p] += 1
    
    for p in zip(xs, ys):
        if d[p] != 1:
            break
    else:
        return True
    return False

validate_blocks([])

False

In [69]:
validate_blocks(blocks)

True

In [70]:
# 問題のブロックの制約を追加

for block in blocks:
    prob += pulp.lpSum([v * choices[v][x][y] for v in numbers for x, y in block.positions]) == block.sum_number

In [71]:
prob

PlusPuzzle:
MINIMIZE
0
SUBJECT TO
_C1: Cell_1_1_1 + Cell_2_1_1 + Cell_3_1_1 + Cell_4_1_1 + Cell_5_1_1 = 1

_C2: Cell_1_2_1 + Cell_2_2_1 + Cell_3_2_1 + Cell_4_2_1 + Cell_5_2_1 = 1

_C3: Cell_1_3_1 + Cell_2_3_1 + Cell_3_3_1 + Cell_4_3_1 + Cell_5_3_1 = 1

_C4: Cell_1_4_1 + Cell_2_4_1 + Cell_3_4_1 + Cell_4_4_1 + Cell_5_4_1 = 1

_C5: Cell_1_5_1 + Cell_2_5_1 + Cell_3_5_1 + Cell_4_5_1 + Cell_5_5_1 = 1

_C6: Cell_1_1_2 + Cell_2_1_2 + Cell_3_1_2 + Cell_4_1_2 + Cell_5_1_2 = 1

_C7: Cell_1_2_2 + Cell_2_2_2 + Cell_3_2_2 + Cell_4_2_2 + Cell_5_2_2 = 1

_C8: Cell_1_3_2 + Cell_2_3_2 + Cell_3_3_2 + Cell_4_3_2 + Cell_5_3_2 = 1

_C9: Cell_1_4_2 + Cell_2_4_2 + Cell_3_4_2 + Cell_4_4_2 + Cell_5_4_2 = 1

_C10: Cell_1_5_2 + Cell_2_5_2 + Cell_3_5_2 + Cell_4_5_2 + Cell_5_5_2 = 1

_C11: Cell_1_1_3 + Cell_2_1_3 + Cell_3_1_3 + Cell_4_1_3 + Cell_5_1_3 = 1

_C12: Cell_1_2_3 + Cell_2_2_3 + Cell_3_2_3 + Cell_4_2_3 + Cell_5_2_3 = 1

_C13: Cell_1_3_3 + Cell_2_3_3 + Cell_3_3_3 + Cell_4_3_3 + Cell_5_3_3 = 1

_C14: Cell_1_

In [72]:
s = prob.solve()

In [73]:
print(pulp.LpStatus[s])

Optimal


In [74]:
for y in ys:
    for x in xs:
        for v in numbers:
            if choices[v][x][y].value() == 1:
                print(v, end=" ")
    print()

2 1 5 4 3 
1 5 4 3 2 
5 4 3 2 1 
3 2 1 5 4 
4 3 2 1 5 
